In [2]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tqdm import tqdm
import tensorflow as tf
from glob import glob
import numpy as np
import json
import os

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print("Tensorflow:",tf.__version__)

Tensorflow: 2.1.0


In [4]:
#输入大小是448*448
H,W = 448,448
grid_size  = [H//64,W//64]
#每个网格预测9个bbox
nboxes = 9
classes = ['bike', 'bus', 'car', 'motor', 'person', 'rider',
           'traffic light', 'traffic sign', 'train', 'truck']
class_map = {k:idx for idx,k in enumerate(classes)}
nclasses = len(classes)
#输出向量的形状，列表相加是[7,7]+[30] = [7,7,30]
output_shape = grid_size + [nboxes*5+nclasses]

In [ ]:
def compute_iou(boxes1,boxes2):
    '''
    输入参数：bbox1，bbox2，形如[x,y,w,h],两个待计算IOU的bbox
    输出参数: 两个bbox的iou
    '''
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0, #最左上点的x坐标
                         boxes1[..., 1] - boxes1[..., 3] / 2.0, #最左上点的y坐标
                         boxes1[..., 0] + boxes1[..., 2] / 2.0, #最右下点的x坐标
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],#最右下点的y坐标
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)
    #lu为x坐标中更偏左上的那一个
    lu = tf.maximum(boxes1_t[...,:2],boxes2_t[...,:2])
    #rd为y坐标中更偏右下的那一个
    rd = tf.minimum(boxes1_t[...,2:],boxes2_t[...,2:])
    
    intersection = tf.maximum(0.0,rd-lu)
    #计算交叉部分的面积
    inter_square = intersection[...,0]*intersection[...,1]
    
    square1 = boxes1[...,2]*boxes1[...,3]
    square2 = boxes2[...,2]*boxes2[...,3]
    
    union_square = tf.maximum(square1+square2-inter_square,1e-10)
    return tf.clip_by_value(inter_square / union_square,0.0,1.0)